# ORTHOGONAL POLYNOMIAL DENSITY ESTIMATION
## Preliminaries
### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from uuid import uuid4

from lpde.geometry import WidthOf, Window, PointAt, BoundingBox, Mapper, Grid
from lpde.estimators import ParallelEstimator
from lpde.estimators.datatypes import Event, Degree, Action
from lpde.producers import MockParams
from lpde.producers.distributions import gaussian

### Notebook settings

In [3]:
%matplotlib qt5

## Density Estimation
### Initialize

In [4]:
legendre_width = WidthOf(1.8)

center = PointAt(51.4, 35.7)
window = Window(0.3, 0.2)
bounds = BoundingBox(center, window)

mapper = Mapper(bounds, legendre_width)

degree = Degree(20, 20)
params = MockParams(20, 1000, gaussian)
density = ParallelEstimator(degree, mapper, params)

action = Action.ADD
point = PointAt(51.4, 35.7)
event = Event(uuid4(), action, point)

grid = Grid(100, 100)

### Start, check, and stop

In [5]:
density.controller.start(1, 1.0)

In [10]:
density.controller.alive

{'minimizers': (False,),
 'producer': False,
 'smoother': False,
 'transformer': False}

In [11]:
density.controller.open

{'coefficients': False, 'events': False, 'phi': False}

In [12]:
density.controller.qsize

{'coefficients': None, 'events': None, 'phi': None}

In [ ]:
density.update_with(event)

In [9]:
density.controller.stop()

### Plot final density

In [19]:
from matplotlib.colors import ListedColormap

In [24]:
old_cmap = plt.cm.viridis
new_cmap = old_cmap(np.arange(old_cmap.N))
new_cmap[:, -1] = np.linspace(0, 1, old_cmap.N)
new_cmap = ListedColormap(new_cmap)

In [25]:
fig_size_x = 8

fig, ax = plt.subplots(figsize=(fig_size_x, fig_size_x*bounds.aspect))
ax.set(xlabel=r'$x$', ylabel=r'$y$')
contour = ax.imshow(density.on_grid,
                    cmap=new_cmap,
                    alpha=1.0,
                    extent=bounds.x_range+bounds.y_range,
                    origin='lower',
                    aspect=bounds.aspect,
                    animated=True)
cbar = plt.colorbar(contour, ax=ax, label=r'$n(x)$', fraction=0.031)
fig.tight_layout()

def update_figure(*args):
    contour.set_data(density.on_grid)
    contour.autoscale()
    return contour,

animation = FuncAnimation(fig, update_figure, interval=50, blit=True)
plt.show()

In [ ]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.tile_providers import STAMEN_TONER, STAMEN_TERRAIN

bound = 20000000 # meters
fig = figure(tools='pan, wheel_zoom', x_range=(-bound, bound), y_range=(-bound, bound))
fig.axis.visible = False
fig.add_tile(STAMEN_TERRAIN)
output_file("stamen_toner_plot.html")
show(fig)

In [ ]:
from bokeh.plotting import figure, show, output_file

p = figure(x_range=bounds.x_range, y_range=bounds.y_range)

# must give a vector of image data for image parameter
p.image(image=[n_hat],
        x=bounds.x_range[0],
        y=bounds.y_range[0],
        dw=bounds.window[0],
        dh=bounds.window[1],
        palette="Spectral11",
        alpha=0.8)

output_file("image.html", title="image.py example")

show(p)  # open a browser